In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

import os
from custom_datasets.rollout_push_any import RolloutPushAnyMemDataset
from custom_datasets.pusht import PushTDataset
from concurrent.futures import ThreadPoolExecutor, wait
from datasets import Dataset
from lerobot.memory_dataset import MemoryDataset

import re
from pathlib import Path
import zarr
import numpy as np
import torch
import torch.nn.functional as F
import imageio
from IPython.display import Video

from typing import Dict
import time

In [2]:
root_path = Path('/home/sm/Datasets/lerobot/pusht')
data_path = root_path / 'episodes.pth'

In [3]:
data_dict = torch.load(data_path, map_location=torch.device('cpu'))

In [4]:
data_dict.keys()

dict_keys(['action', 'episode_index', 'frame_index', 'next.done', 'next.reward', 'next.success', 'observation.state', 'timestamp', 'observation.image', 'index'])

In [5]:
dataset = RolloutPushAnyMemDataset('/home/sm/Datasets/lerobot/pusht')

In [10]:
memory_dataset = MemoryDataset(data_dict)

In [7]:
len(dataset)

2000

In [11]:
def get_episode(episode_idx):
    from_idx, to_idx = dataset._get_episode_from_to(0)
    return memory_dataset[from_idx:to_idx]

In [24]:
d = get_episode(2)
d.keys()

dict_keys(['action', 'episode_index', 'frame_index', 'next.done', 'next.reward', 'next.success', 'observation.state', 'timestamp', 'observation.image', 'index'])

In [26]:
d = {k: v for k, v in d.items() if k != 'index'}

In [27]:
d.keys()

dict_keys(['action', 'episode_index', 'frame_index', 'next.done', 'next.reward', 'next.success', 'observation.state', 'timestamp', 'observation.image'])

In [16]:
episode_path = root_path / 'episodes'

In [33]:
import re

def get_max_episode_id(episode_path):
    pattern = re.compile(r"episode_(\d+)\.pth")
    max_idx = -1
    for filename in os.listdir(episode_path):
        match = pattern.match(filename)
        if match:
            idx = int(match.group(1))
            max_idx = max(max_idx, idx)
    return max_idx

In [34]:
get_max_episode_id(episode_path)

1999

In [35]:
def generate_episode_data(episode_path):
    def gen():
        num_episodes = get_max_episode_id(episode_path) + 1
        total_frames = 0
        for episode_idx in range(num_episodes):
            d = torch.load(episode_path / f'episode_{episode_idx}.pth')
            episode_num_frames = len(d['frame_index'])
            d['index'] = torch.arange(total_frames, total_frames + episode_num_frames, 1)
            total_frames += episode_num_frames
            yield d
    return gen

In [36]:
dataset = Dataset.from_generator(generator=generate_episode_data(episode_path))

Generating train split: 0 examples [00:00, ? examples/s]

In [49]:
ep_idx = 3
min(dataset[ep_idx]['index']), max(dataset[ep_idx]['index'])

(813, 1083)